In [2]:
import pandas as pd

In [3]:
exams_preproc = pd.read_csv('../exams_preproc.csv', sep=';')

In [4]:
print(exams_preproc.head().to_latex(index=False))

\begin{tabular}{lllrrlr}
\toprule
Id\_studente & codice\_esame &  data\_esame &  voto &  crediti &                                    descrizione &  Semester \\
\midrule
          A &      B006807 &  2015-01-16 &    24 &        6 &                                ALGEBRA LINEARE &         2 \\
          A &      B006800 &  2014-07-17 &    29 &       12 &                     ALGORITMI E STRUTTURE DATI &         1 \\
          A &      B006801 &  2014-06-10 &    30 &       12 &  ANALISI I: CALCOLO DIFFERENZIALE ED INTEGRALE &         1 \\
          A &      B006808 &  2017-01-23 &    18 &        6 &         ANALISI II: FUNZIONI DI PIU' VARIABILI &         6 \\
          A &      B006802 &  2014-06-12 &    25 &       12 &                 ARCHITETTURE DEGLI ELABORATORI &         1 \\
\bottomrule
\end{tabular}



In [5]:
grouped = exams_preproc.groupby(['Id_studente', 'descrizione'])['Semester'].mean()

In [6]:
id_to_exam_name = {  1: 'ALGEBRA LINEARE',
                     2: 'ANALISI I: CALCOLO DIFFERENZIALE ED INTEGRALE',
                     3: 'PROGRAMMAZIONE',
                     4: "ANALISI II: FUNZIONI DI PIU' VARIABILI",
                     5: 'FISICA GENERALE',
                     6: 'MATEMATICA DISCRETA E LOGICA',
                     7: 'METODOLOGIE DI PROGRAMMAZIONE',
                     8: 'PROGRAMMAZIONE CONCORRENTE',
                     9: 'SISTEMI OPERATIVI',
                     10: 'BASI DI DATI E SISTEMI INFORMATIVI',
                     11: 'ALGORITMI E STRUTTURE DATI',
                     12: "CALCOLO DELLE PROBABILITA' E STATISTbICA",
                     13: 'ARCHITETTURE DEGLI ELABORATORI' }
exam_name_to_id = {exam_name: id_ for id_, exam_name in id_to_exam_name.items()}
set_id_exams = set(id_to_exam_name)

In [21]:
print("\n".join([f"{k} -> {v}" for k,v in id_to_exam_name.items()]))

1 -> ALGEBRA LINEARE
2 -> ANALISI I: CALCOLO DIFFERENZIALE ED INTEGRALE
3 -> PROGRAMMAZIONE
4 -> ANALISI II: FUNZIONI DI PIU' VARIABILI
5 -> FISICA GENERALE
6 -> MATEMATICA DISCRETA E LOGICA
7 -> METODOLOGIE DI PROGRAMMAZIONE
8 -> PROGRAMMAZIONE CONCORRENTE
9 -> SISTEMI OPERATIVI
10 -> BASI DI DATI E SISTEMI INFORMATIVI
11 -> ALGORITMI E STRUTTURE DATI
12 -> CALCOLO DELLE PROBABILITA' E STATISTICA
13 -> ARCHITETTURE DEGLI ELABORATORI


### Conversione in sequenza temporale del dataset preprocessato

In [7]:
from collections import defaultdict
exams_per_student = defaultdict(list)
for (id_stud, id_exam), sem in grouped.iteritems():
    id_exam = exam_name_to_id[id_exam]
    exams_per_student[id_stud].append((sem, id_exam))
for id_stud in exams_per_student:
    exams_per_student[id_stud].sort(key=lambda sem__id_exam: sem__id_exam[0])

In [8]:
def flat_semesters(sequence):
    result = defaultdict(list)
    for sem, id_exam in sequence:
        result[sem].append(id_exam)
    result = list(result.items())
    result.sort(key=lambda pair: pair[0])
    return [events for i, events in result]

seq_temp_per_student = [flat_semesters(v) for k, v in exams_per_student.items()]

In [22]:
import string
id_studs = []
str_seq_temps = []
for i, events in enumerate(seq_temp_per_student):
    events = "<" + str(events).replace("[", "{").replace("]", "}")[1:-1] + ">"
    id_studs.append(string.ascii_uppercase[i])
    str_seq_temps.append(events)
#     print(f"Student {i+1} exams per semester: <{events[1:-1]}>")
df_seq_temps = pd.DataFrame({"Id studente":id_studs, "Sequenza temporale":str_seq_temps})
print(df_seq_temps)


   Id studente                                 Sequenza temporale
0            A           <{11, 2, 13, 3}, {1, 7, 8}, {4, 10, 12}>
1            B      <{11, 2, 3}, {7}, {4, 10}, {1, 13}, {6}, {9}>
2            C                   <{11, 3}, {10, 7}, {13}, {8, 9}>
3            D                     <{2, 3}, {1, 11}, {4, 13, 12}>
4            E                                        <{11}, {2}>
5            F                  <{11, 2, 3}, {1, 13}, {10, 6, 8}>
6            G   <{11, 2, 6, 3}, {1, 13, 12, 7}, {4, 10, 8}, {9}>
7            H        <{11, 2, 13, 6, 3}, {1, 8}, {4, 10, 12, 9}>
8            I                         <{11, 2}, {1, 12, 8}, {4}>
9            J              <{11, 2, 6, 3}, {1, 4, 10, 12, 5, 8}>
10           K  <{11, 2, 13, 6, 3}, {1, 12, 7, 8}, {4, 10, 5, 9}>
11           L  <{11, 2, 13, 6, 3}, {1, 12, 7, 8}, {4, 10, 5, 9}>
12           M                         <{11, 3}, {2, 7}, {4, 10}>
13           N     <{11, 2, 3}, {1, 13, 12, 7, 8}, {4, 10, 6, 9}>
14        

### Serializzazione strutture dati python

In [11]:
with open('../seq_temp_python.txt', mode='w') as f:
    f.write(str(seq_temp_per_student))
with open('../id_exams_python.txt', mode='w') as f:
    f.write(str(id_to_exam_name))

### Esportazione per weka

In [10]:
def sort_by_abs_value(lst):
    return sorted(lst, key=abs)

def negative(lst):
    return [-x for x in lst]

In [11]:
import string
seq_temp_weka = [[sort_by_abs_value(negative(set_id_exams.difference(set(event))) + event) for event in events] for events in seq_temp_per_student]
seq_temp_weka_str = [[[('n' if x < 0 else '') + string.ascii_uppercase[abs(x)-1] for x in event] for event in events] for events in seq_temp_weka]

In [31]:
input_weka = []
for i, events in enumerate(seq_temp_weka_str):
    for event in events:
        input_weka.append(f"{i+1}," + str(event).replace("[", "").replace("]", "").replace(" ", "").replace("'", ""))

In [32]:
with open('../input_weka_gsp.txt', mode='w') as f:
    f.write("\n".join(input_weka))

In [13]:
with open('../seq_temp_binary.txt', mode='w') as f:
    f.write(str(seq_temp_weka))

### Esportazione per SPMF

In [33]:
def without_squares(lst):
    return str(lst)[1:-1]

def events_to_spmf_format(events):
    event_spmf = (without_squares(event).replace(",", "") for event in events)
    return " -1 ".join(event_spmf)

def to_spmf_input(seq_temp):
    return " -2\n".join(map(events_to_spmf_format, seq_temp)) + " -2"

In [34]:
with open('../spfm_gsp_input.txt', mode='w') as f:
    f.write(to_spmf_input(seq_temp_per_student))